# Spermosphere logbook

Initialize Qiime2:

In [1]:
source activate qiime2-2019.10

(qiime2-2019.10) 

: 1

Export ASV table stats visualization:

In [ ]:
qiime feature-table summarize \
  --i-table table.qza \
  --o-visualization table.qzv \
  --m-sample-metadata-file metadata.tsv

## Spermosphere taxonomic classification

We created the folder `classifier-spermosphere/` in the BIOS server, inside this folder is the file `rep-seqs.qza` containing reads from spermosphere and the classifier `classifier.qza`.

Then we ran the classifier:

```bash
screen
time qiime feature-classifier classify-sklearn \
  --i-classifier classifier.qza \
  --i-reads rep-seqs.qza \
  --o-classification taxonomy.qza
```

Export taxonomy to a TSV file:

In [ ]:
qiime tools export \
    --input-path taxonomy-spermosphere/taxonomy.qza \
    --output-path taxonomy-spermosphere/

## 2020-03-04 Phylogenetic tree from taxonomic classification

Using the files `table.qza` and `taxonomy.qza` created by the Qiime2 classifier using Silva seqs as training data we ran the following command:

```bash
qiime taxa collapse \
      --i-table table.qza \
      --i-taxonomy taxonomy.qza \
      --p-level 6 \ # or whatever level of taxonomy you want
      --output-dir taxtable/
```

That info was in [this page](https://forum.qiime2.org/t/how-to-make-classic-otu-table-with-qiime2/3612/3).

In the output folder `taxtable/` was stored a file named `collapsed-table.qza`, this file was decompressed and then the file `feature-table.biom` contained there was converted to `.tsv` runing the next command:

`(qiime2-2019.10) nesper@gojira-E402MA:~/Documentos/genomeseq/taxonomy-spermosphere/phylogenetic-tree/taxtable$ biom convert -i 8f17f216-8ae0-43ba-a691-fbb0428afc4d/data/feature-table.biom -o feature-table-taxonomy.tsv --to-tsv`


We also generated a barplot of the taxa found in the classification using the next command:

```bash
qiime taxa barplot \
  --i-table table.qza \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file metadatos.tsv \
  --o-visualization taxa-bar-plots.qzv
```
  
This was done following the instructions in [this link](https://chmi-sops.github.io/mydoc_qiime2.html#step-9-assign-taxonomy).

The output file was visualized with the following command:

    qiime tools view taxa-bar-plots.qzv

![taxa-bar-plot-spermosphere](taxonomy-spermosphere/taxa-bar-plot-spe.png)

**Fig 1.** Taxonomical bar graphs showing the phyla of bacteria found in the different samples. Colors are presented from the top to the bottom of the graph (A: Ancestral, M: Modern, Soil: Uncultured soil).

## 2020-03-06 Phylogenetic tree

After aligning the spermosphere sequences, we performed the masking to remove non-informative positions in the alignment and create more efficiently the phylogeny. We used the following command in the BIOS server:

```bash
qiime alignment mask \
  --i-alignment aligned-rep-seqs-spe.qza \
  --o-masked-alignment masked-aligned-rep-seqs-spermosphere.qza
```

We then made the phylogenetic inference:

```bash
qiime phylogeny fasttree \
  --i-alignment masked-aligned-rep-seqs-spermosphere.qza \
  --o-tree fasttree-tree-spermosphere.qza```
  
### Fasttree to Newick

To convert the phylogenetic tree generated in Qiime2 by mafft and then use fasttree, we used the next command:

```bash
qiime tools export --input-path fasttree-tree-spermosphere.qza --output-path newick-tree-spe
```

In the folder `newick-tree-spe/` is stored the file `tree.nwk` with the info from the phylogenetic tree.

**From here we will use the abreviation *spe* to denote files related with spermosphere data.**

## iTOL tree

The fasttree phylogenetic tree was uploaded to iTOL along with the `taxonomy.qza` file.

We also uploaded `table.qza` containing the abundance information.

## 2020-03-25 Composition analysis (ANCOM)

We are also analyzing composition in order to know which ASVs are differentially abundant between samples. We ran the following commands:

```bash
(qiime2-2019.10) jeperezj@r1masterl01:~/jcar/spermosphere/ancom-composition-analysis$ qiime composition add-pseudocount \
  --i-table table.qza \
  --o-composition-table comp-table-spe.qza
  
(base) jeperezj@r1masterl01:~/jcar/rhizosphere/ancom-composition-analysis$ qiime composition ancom \
    --i-table comp-table-spe.qza \
    --m-metadata-file metadatos.tsv \
    --m-metadata-column Plant_status \
    --o-visualization ancom-plant-status-spe.qzv```

This last output file was transferred to my laptop:

```bash
(base) nesper@gojira-E402MA:~/Documentos/genomeseq/vpnbios-jeperezj$ scp jeperezj@door1vpn.bios.co:~/jcar/spermosphere/ancom-composition-analysis/ancom-plant-status-spe.qzv ../spermosphere/ancom-composition-analysis/
```
ANCOM visualization:

In [ ]:
qiime tools view ancom-composition-analysis/ancom-plant-status-spe.qzv

This is the output visualization with taxa:

![Ancom volcano plot spermosphere with taxa](ancom-composition-analysis/ancom-volcano-plot-spe-taxa.png)
**Fig 2.** ANCOM Volcano plot.

These are the taxonomic entities that were found to be differentially abundant between samples:

D_0__Bacteria;D_1__Actinobacteria;D_2__Actinobacteria;D_3__Micrococcales;D_4__Micrococcaceae;D_5__Paenarthrobacter (0.94648)
D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Xanthomonadales;D_4__Xanthomonadaceae;D_5__Luteimonas (0.73401)

## ANCOM with only cultured soil data

As we expect to find a big difference between microbial composition in cultured vs. uncultured soil, and the ANCOM is best detecting differential abundance between similar samples, it is better to create a data set with only cultured soil and compare Modern and Ancestral samples:

```bash
(qiime2-2019.10) jeperezj@r1masterl01:~/jcar/spermosphere$ qiime feature-table filter-samples \
    --i-table table.qza \
    --m-metadata-file metadatos.tsv \
    --p-where "[Plant_status]!='Soil'" \
    --o-filtered-table cultured-soil-table-spe.qza

(qiime2-2019.10) jeperezj@r1masterl01:~/jcar/spermosphere/ancom-composition-analysis/only-cultured-soil-ancom$ qiime composition add-pseudocount \
    --i-table ../../cultured-soil-table-spe.qza \
    --o-composition-table comp-table-cultured-spe.qza

(qiime2-2019.10) jeperezj@r1masterl01:~/jcar/spermosphere/ancom-composition-analysis/only-cultured-soil-ancom$ time qiime composition ancom \
    --i-table comp-table-cultured-spe.qza  \
    --m-metadata-file ../metadatos.tsv  \
    --m-metadata-column Plant_status  \
    --o-visualization ancom-plant-status-cultured-spe.qzv
```

### ANCOM grouping ASVs by family

We also grouped ASVs by family and ran ANCOM with only cultured soil at family level:

```bash
(qiime2-2019.10) jeperezj@r1masterl01:~/jcar/spermosphere/ancom-composition-analysis/collapsed-to-family$ qiime taxa collapse \
    --i-table cultured-soil-table-spe.qza \
    --i-taxonomy taxonomy.qza \
    --o-collapsed-table table-family-level-spe.qza \
    --p-level 5
    
(qiime2-2019.10) jeperezj@r1masterl01:~/jcar/spermosphere/ancom-composition-analysis/collapsed-to-family$ qiime composition add-pseudocount \
    --i-table table-family-level-spe.qza \
    --o-composition-table comp-table-famlvl-spe.qza

(qiime2-2019.10) jeperezj@r1masterl01:~/jcar/spermosphere/ancom-composition-analysis/collapsed-to-family$ time qiime composition ancom \
    --i-table comp-table-famlvl-spe.qza \
    --m-metadata-file metadatos.tsv \
    --m-metadata-column Plant_status \
    --o-visualization ancom-famlvl-cult-spe.qzv
```
**Nota:** This analysis took only 22 seconds! Previous ANCOM analysis took more than 15 hours.

# 2020-04-01 Reunión Meet

Por hacer:
- Ayudarle a Estefa con análisis de abundancia diferencial en R con el paquete MetagenomeSeq.
- Seguir con el ANCOM.
- Leer paper ANCOM.
- Seguir con el árbol filogenético en iTOL.

# 2020-04-05 Gneiss composition analysis

We will perform a composition analysis with Gneiss and compare its results with those from ANCOM. To do that we will follow [this tutorial](https://docs.qiime2.org/2020.2/tutorials/gneiss/).

```bash
(qiime2-2019.10) jeperezj@r1masterl01:~/jcar/spermosphere/gneiss-composition-analysis$ qiime gneiss correlation-clustering \
    --i-table ../table.qza \
    --o-clustering hierarchy-spe.qza
    
qiime gneiss dendrogram-heatmap \
  --i-table table.qza \
  --i-tree hierarchy-spe.qza \
  --m-metadata-file metadatos.tsv \
  --m-metadata-column Plant_status \
  --p-color-map seismic \
  --o-visualization heatmap-spe.qzv

```

# Dudas para reunión 2020-04-07
En el material suplementario del paper de ANCOM (Mandal et al., 2015) decía:
> As commonly done [5], we restricted the analysis to taxa that are present in at least 25%
of the samples. This is done because low frequency OTUs are often thought to be difficult to interpret statistically.

La pregunta es ¿hacemos lo mismo con nuestros datos?

## 2020-04-16 Installing PhyloToAST

The PhyloToAST project is a collection of python scripts that modifies the original QIIME pipeline. We will use it to produce a nice phylogenetic tree with information of abundances in the different samples.

Visit [PhyloToAST web page](https://phylotoast.readthedocs.io/en/latest/index.html) for more information.

Install PhyloToAST:

In [ ]:
pip3 install phylotoast

We will use specifically the scrip `iTol.py`. More information about the script [here](https://phylotoast.readthedocs.io/en/latest/scripts/iTol.html).

In [ ]:
iTol.py -h

Install required packages:

In [ ]:
pip3 install cython
pip3 install biom-format

Export phylogenetic tree generated in Qiime2 in Newick format:

In [ ]:
qiime tools export \
    --input-path phylogenetic-tree/fasttree-tree-spermosphere.qza \
    --output-path phylogenetic-tree/

Export feature table to Biom format:

In [ ]:
qiime tools export \
    --input-path table.qza \
    --output-path ./

For some reason probably related to conda environment `iTol.py` doesn't work when Qiime2 is active. So be sure to run it without activating Qiime2 (for example by restarting the kernel in this notebook).

In [ ]:
iTol.py \
    -i feature-table.biom \
    -m metadatos.tsv \
    -t phylogenetic-tree/tree.nwk \
    -e tree \
    -o itol-table \
    -c Plant_status \
    -a NMRA \
    --stabilize_variance

**Note:** The previous error could not be solved, so I abandoned this approach.

In [this forum](https://forum.qiime2.org/t/loading-greengenes-tree-in-itol/2319/10) Ivica Letunic, creator of iTol, recommends to create a zip file with Qiime2 artifacts (`tree.qza`,`taxonomy.qza` and `taxa-bar-plots.qzv`)  and upload it to iTol. Let's see how it works:

In [ ]:
zip itol phylogenetic-tree/fasttree-tree-spermosphere.qza taxonomy-spermosphere/taxonomy.qza taxonomy-spermosphere/phylogenetic-tree/taxa-bar-plots.qzv

In the same forum Letunic reported he had not implemented the uploading of files as zip but uploading each `.qza` to iTol wich was what I have done, in exception of `taxa-bar-plots.qzv` because iTol doesn't support it... so we are were we started.

## 2020-04-17 Filtering of low abundance features

In order to produce a more clean phylogenetic tree we are going to remove features with less than 5% of relative abundance. We need to know the total abundance of the features in order to know which features to remove:

In [ ]:
qiime tools view table.qzv

We now proceed to filter the feature table and the rep-seqs accordingly:

In [13]:
mkdir filtered-data

qiime feature-table filter-features \
    --i-table table.qza \
    --p-min-frequency 527 \
    --o-filtered-table filtered-data/filtered-table.qza

qiime feature-table filter-seqs \
  --i-data rep-seqs.qza \
  --i-table filtered-data/filtered-table.qza \
  --o-filtered-data filtered-data/filtered-rep-seqs.qza

(qiime2-2019.10) (qiime2-2019.10) Saved FeatureTable[Frequency] to: filtered-data/filtered-table.qza
(qiime2-2019.10) (qiime2-2019.10) Saved FeatureData[Sequence] to: filtered-data/filtered-rep-seqs.qza
(qiime2-2019.10) 

: 1

Produce the phylogenetic tree with filtered data:

In [14]:
cd filtered-data

qiime alignment mafft \
  --i-sequences filtered-rep-seqs.qza \
  --o-alignment aligned-filtered-rep-seqs.qza

qiime alignment mask \
  --i-alignment aligned-filtered-rep-seqs.qza \
  --o-masked-alignment masked-aligned-filtered-rep-seqs.qza

qiime phylogeny fasttree \
  --i-alignment masked-aligned-filtered-rep-seqs.qza \
  --o-tree unrooted-tree-filtered.qza
  
cd ../

(qiime2-2019.10) (qiime2-2019.10) Saved FeatureData[AlignedSequence] to: aligned-filtered-rep-seqs.qza
(qiime2-2019.10) (qiime2-2019.10) Saved FeatureData[AlignedSequence] to: masked-aligned-filtered-rep-seqs.qza
(qiime2-2019.10) (qiime2-2019.10) Saved Phylogeny[Unrooted] to: unrooted-tree-filtered.qza
(qiime2-2019.10) 

: 1

Group samples by Plant_status:

In [5]:
qiime feature-table group \
    --i-table filtered-data/filtered-table.qza \
    --p-axis sample \
    --m-metadata-file metadata.tsv \
    --m-metadata-column Plant_status \
    --p-mode sum \
    --o-grouped-table filtered-data/grouped-filtered-table.qza

Saved FeatureTable[Frequency] to: filtered-data/grouped-filtered-table.qza
(qiime2-2019.10) 

: 1

Transform absolute abundance data in relative abundance data:

In [6]:
cd filtered-data

(qiime2-2019.10) 

: 1

In [8]:
qiime feature-table relative-frequency \
--i-table grouped-filtered-table.qza \
--o-relative-frequency-table rel-grouped-filtered-table.qza

Saved FeatureTable[RelativeFrequency] to: rel-grouped-filtered-table.qza
(qiime2-2019.10) 

: 1